In [39]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [40]:
import torch

num_gpus = torch.cuda.device_count()
print(f"Available GPUs: {num_gpus}")

for i in range(num_gpus):
    print(f"Device ID: {i}, GPU Name: {torch.cuda.get_device_name(i)}")


Available GPUs: 0


In [41]:
from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device

In [42]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# hf_access_token = os.environ.get('HF_ACCESS_TOKEN')

In [43]:
# model_path = 'HuggingFaceH4/zephyr-7b-alpha'
model_path='/media/d1/huggingface.co/models/meta-llama/Llama-2-7b-chat-hf/'
# model_path='meta-llama/Llama-2-7b-chat-hf' 

In [44]:
# FIXME: Should be float16 instead of float32
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32, device_map="auto")
# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32, device_map="auto", token=hf_access_token)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
prompt = "Please list 17 U.S. politicians that have been active since 1980. Format the response as a simple, un-numbered comma separated list with only full names."

In [46]:
prompt = "List 20 famous U.S. politicians that also have a famous blood relative in politics. List only their names. Only include cases where both the politician and the corresponding relative have had political careers that have been active since the 1980s. Only include cases where both the politician and the corresponding relative have the same surname."

In [47]:
prompt

'List 20 famous U.S. politicians that also have a famous blood relative in politics. List only their names. Only include cases where both the politician and the corresponding relative have had political careers that have been active since the 1980s. Only include cases where both the politician and the corresponding relative have the same surname.'

In [25]:
model_input = tokenizer(prompt, return_tensors="pt").to(device)

# Perform inference
model.eval()
with torch.no_grad():
    response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

# Extract and display the response
response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
print("User Prompt:")
print(prompt)
print("\nResponse:")
print(response_text.strip())

User Prompt:
For each politicians returned in the list above, please list a corresponding blood relative that is also in politics.

Response:
For example, for Angela Merkel, her brother Horst Merkel is also in politics.

Note: This list only includes politicians who are directly related to the politicians listed above (i.e. siblings, parents, children).

1. Angela Merkel - Horst Merkel (brother)
2. Emmanuel Macron - None
3. Justin Trudeau - None
4. Boris Johnson - None
5. Scott Morrison - None
6. Jair Bolsonaro - None
7. Narendra Modi - None
8. Xi Jinping - None
9. Vladimir Putin - None
10. Recep Tayyip Erdogan - None


In [54]:
from entity_extraction import EntityExtraction

def get_political_names(prompt, minResults=5, retryCnt=3):
    model_input = tokenizer(prompt, return_tensors="pt").to(device)
    # Perform inference
    model.eval()
    with torch.no_grad():
        response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

    # Extract and display the response
    response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
    extractor = EntityExtraction()

    names = []
    while True:
        names = extractor.get_full_names(response_text)
        if len(names) < minResults and retryCnt > 0:
            retryCnt =- 1
            continue
        break

    return names


In [55]:
names = get_political_names(prompt)
print("names: ", names)    




names:  ['Joe Biden', 'John F. Kennedy', 'Robert F. Kennedy', 'Ted Kennedy', 'Barack Obama', 'Bill Clinton', 'Hillary Clinton', 'George H.W', 'Bush', 'George W. Bush', 'Jeb Bush', 'Jimmy Carter', 'Rosalynn Carter', 'Al Gore', 'Tipper Gore', 'John McCain', 'Meghan McCain', 'Lindsey Graham', 'Bob Graham', 'Michelle Obama']


In [26]:
# from textattack.nlp import EntityExtraction
from entity_extraction import EntityExtraction


extractor = EntityExtraction()
names = extractor.get_full_names(text=response_text)
if len(names) > 0:
    for name in names:
        name = ' '.join(name.split())
        print(f"name: \"{name}\"")
        # print(name)


print("names: ", names)

name: "Angela Merkel"
name: "Horst Merkel"
name: "Angela Merkel"
name: "Horst Merkel"
name: "Emmanuel Macron"
name: "Justin Trudeau"
name: "Boris Johnson"
name: "Scott Morrison"
name: "Jair Bolsonaro"
name: "Narendra Modi"
name: "Vladimir Putin"
name: "Recep Tayyip Erdogan"
names:  ['Angela Merkel', 'Horst Merkel', 'Angela Merkel', 'Horst Merkel', 'Emmanuel Macron', 'Justin Trudeau', 'Boris Johnson', 'Scott Morrison', 'Jair Bolsonaro', 'Narendra Modi', 'Vladimir Putin', 'Recep Tayyip Erdogan']


In [19]:
prompt = "For each politicians returned in the list above, please list a corresponding blood relative that is also in politics."

In [20]:
model_input = tokenizer(prompt, return_tensors="pt").to(device)

# Perform inference
model.eval()
with torch.no_grad():
    response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

# Extract and display the response
response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
print("User Prompt:")
print(prompt)
print("\nResponse:")
print(response_text.strip())

User Prompt:
For each politicians returned in the list above, please list a corresponding blood relative that is also in politics.

Response:
For example, for Angela Merkel, her brother Horst Merkel is also in politics.

Note: I will only consider blood relatives, not spouses or partners.
